# Simpler Chunking
- Here we switch to a more coherent tutorial that's closer to what we want. The link is commented below
- This has much simpler (and equally more effecient) chunking. It simply tokenizes the whole input and splits them up to the maximum length
- This is the closest attempt byfar but just needed more time

In [1]:
#https://www.mim.ai/fine-tuning-bert-model-for-arbitrarily-long-texts-part-1/

from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-uncased")

c:\Users\Adam\anaconda3\envs\FlashGov_ML_Server\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Adam\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [3]:
from datasets import load_dataset

imdb = load_dataset("imdb")

long_review = imdb["test"]["text"][21132]
number_of_words = len(long_review.split())
print(f"The review has {number_of_words} words.")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to C:/Users/Adam/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The review has 2278 words.


In [4]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('fabriceyhc/bert-base-uncased-imdb')
model = BertForSequenceClassification.from_pretrained('fabriceyhc/bert-base-uncased-imdb')

In [5]:
tokens = tokenizer(long_review, add_special_tokens=False, truncation=False, return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (3155 > 512). Running this sequence through the model will result in indexing errors


## Token tesing

In [6]:
## tokens example
example = ["the man went to the store and bought a gallon of milk"]
tokens = tokenizer(example, add_special_tokens=False, truncation=False, return_tensors="pt")
tokens


{'input_ids': tensor([[ 1996,  2158,  2253,  2000,  1996,  3573,  1998,  4149,  1037, 25234,
          1997,  6501]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
example = ["the man went to the store and bought a gallon of milk"]
tokens = tokenizer(example, add_special_tokens=True, truncation=False, return_tensors="pt")
tokens

{'input_ids': tensor([[  101,  1996,  2158,  2253,  2000,  1996,  3573,  1998,  4149,  1037,
         25234,  1997,  6501,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [77]:
from torch import Tensor

def split_overlapping(tensor: Tensor, chunk_size: int, stride: int, minimal_chunk_length: int) -> list[Tensor]:
    print("in split over. chunk size:", chunk_size)
    """Helper function for dividing 1-dimensional tensors into overlapping chunks."""
    result = [tensor[i : i + chunk_size] for i in range(0, len(tensor), stride)]
    if len(result) > 1:
        # ignore chunks with less than minimal_length number of tokens
        result = [x for x in result if len(x) >= minimal_chunk_length]
    return result

example_tensor = tokens["input_ids"][0]
example_tensor


# test it here
splitted = split_overlapping(example_tensor, chunk_size=5, stride=5, minimal_chunk_length=5)
splitted

in split over. chunk size: 5


[tensor([ 101, 1996, 2158, 2253, 2000]),
 tensor([1996, 3573, 1998, 4149, 1037])]

In [67]:
def add_special_tokens_at_beginning_and_end(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    """
    Adds special CLS token (token id = 101) at the beginning.
    Adds SEP token (token id = 102) at the end of each chunk.
    Adds corresponding attention masks equal to 1 (attention mask is boolean).
    """
    for i in range(len(input_id_chunks)):
        # adding CLS (token id 101) and SEP (token id 102) tokens
        print("in here. id chunks dim:", input_id_chunks[i].size(), Tensor([101]).size())
        input_id_chunks[i] = torch.cat([Tensor([101]), input_id_chunks[i], Tensor([102])])
        # adding attention masks  corresponding to special tokens
        mask_chunks[i] = torch.cat([Tensor([1]), mask_chunks[i], Tensor([1])])

In [10]:
def add_padding_tokens(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    """Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens."""
    for i in range(len(input_id_chunks)):
        # get required padding length
        pad_len = 512 - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([input_id_chunks[i], Tensor([0] * pad_len)])
            mask_chunks[i] = torch.cat([mask_chunks[i], Tensor([0] * pad_len)])

In [11]:
def stack_tokens_from_all_chunks(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> tuple[Tensor, Tensor]:
    """Reshapes data to a form compatible with BERT model input."""
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    return input_ids.long(), attention_mask.int()

In [69]:
# my function

def tokenize_whole_text(text, tokenizer):
    return tokenizer(text, add_special_tokens=False, truncation=False, return_tensors="pt")

def split_tokens_into_smaller_chunks(tokens, chunk_size, stride, minimal_chunk_length):
    chunks = split_overlapping(tokens, chunk_size, stride,  minimal_chunk_length)
    print("chunks: youre a list?", type(chunks), "of tensors?", type(chunks[0]))
    print(chunks)
    print(chunks[0])

    
    masks = []
    ids = []
    for chunk in chunks:
        ids.append(chunk["input_ids"].squeeze())
        masks.append(chunk["attention_mask"].squeeze())
    return masks, ids

In [70]:

def transform_single_text(
    text: str,
    tokenizer,
    chunk_size: int,
    stride: int,
    minimal_chunk_length: int,
    maximal_text_length: False,
) -> tuple[Tensor, Tensor]:
    """Transforms (the entire) text to model input of BERT model."""
    # if maximal_text_length:
        # tokens = tokenize_text_with_truncation(text, tokenizer, maximal_text_length)
    # else:
    tokens = tokenize_whole_text(text, tokenizer)
    input_id_chunks, mask_chunks = split_tokens_into_smaller_chunks(tokens, chunk_size, stride, minimal_chunk_length)
    print("tf?", input_id_chunks[0].size() , "d", mask_chunks[0].size())
    print("types:", type(input_id_chunks), type(mask_chunks))
    print("tf?", len(input_id_chunks) , "d", len(mask_chunks))
    add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks)
    add_padding_tokens(input_id_chunks, mask_chunks)
    input_ids, attention_mask = stack_tokens_from_all_chunks(input_id_chunks, mask_chunks)
    return input_ids, attention_mask

In [78]:
# example
input_ids, attention_mask = transform_single_text(long_review, tokenizer, 510, 510, 1, None)
print("RESULT")
print(input_ids.size())
print(attention_mask.size())

in split over. chunk size: 510
chunks: youre a list? <class 'list'> of tensors? <class 'dict'>
[{'input_ids': tensor([[2045, 1005, 1055,  ..., 1997, 2184, 1012]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}]
{'input_ids': tensor([[2045, 1005, 1055,  ..., 1997, 2184, 1012]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
tf? torch.Size([3155]) d torch.Size([3155])
types: <class 'list'> <class 'list'>
tf? 1 d 1
in here. id chunks dim: torch.Size([3155]) torch.Size([1])
RESULT
torch.Size([1, 3157])
torch.Size([1, 3157])


In [79]:
model_output = model(input_ids, attention_mask)
model_output

RuntimeError: The expanded size of the tensor (3157) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 3157].  Tensor sizes: [1, 512]

In [ ]:
probs = torch.nn.functional.softmax(model_output[0], dim=-1)
probs

In [ ]:


probabilities = probs[:,1]
probabilities



In [ ]:


probabilities.mean()



In [ ]:


probabilities.max()

